# Last mile robots

In [1]:
# libraries are always a good place to start
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns

from Battery import Battery
from DeliveryRobot import DeliveryRobot

from Task import Task

In [2]:
possible_cities = pd.read_csv(r'last-mile-cities.csv')
possible_cities

,city
0,Glasgow
1,Leeds
2,Liverpool
3,Newcastle
4,Belfast
5,Inverness
6,Carlisle
7,Lancaster
8,Derry
9,Wrexham


In [35]:
# choose a city
city = 'Cambridge'
solomon_instance = 'r101'

# get instance and weather file
instance_file = 'C:/Users/user/ML/Optimisation/VRPTW/solomon/augmented/' + city + '_2.5_' + solomon_instance + '.csv'
weather_file = 'C:/Users/user/ML/Optimisation/VRPTW/weather_data/open-meteo/' + city+'-weather.csv'

# get the locations and clean
locations = pd.read_csv(instance_file, usecols=['xcoord.','ycoord.','elevation'])
locations.columns = ['x', 'y', 'elevation']

city+'-weather.csv'
locations['x'] = locations['x']*25
locations['y'] = locations['y']*25
locations['elevation'] = np.round(locations['elevation'], 2)

print(np.max(locations['x']), np.max(locations['y']), np.min(locations['x']), np.min(locations['y']))

locations

1675 1925 50 75


,x,y,elevation
0,875,875,15.65
1,1025,1225,16.84
2,875,425,15.30
3,1375,1125,18.95
4,1375,500,18.71
...,...,...,...
96,550,675,15.25
97,625,525,16.80
98,475,525,13.59
99,500,650,14.44


In [52]:
# read the weather data
weather = pd.read_csv(weather_file)
weather = weather.drop(['date', 'dew_point_2m', 'wind_direction_10m', 'wind_gusts_10m', 'is_day'], axis=1)

weather['day'] = weather['day'].astype(int)
weather['month'] = weather['month'].astype(int)
weather['hour'] = weather['hour'].astype(int)

weather.dtypes


temperature_2m    float64
rain              float64
wind_speed_10m    float64
month               int32
day                 int32
hour                int32
dtype: object

In [53]:
def plot_weather_data(weather):
    import matplotlib.dates as mdates

    weather['Year'] = 2026  # Arbitrary fixed year
    weather['Date'] = pd.to_datetime(weather[['day', 'month', 'Year']], dayfirst=True)

    month_names = {
        1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
        5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
        9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
    }

    plt.figure(figsize=(20, 15))

    for month in range(1, 13):
        ax = plt.subplot(4, 3, month)
        month_data = weather[weather['Month'] == month]

        # Bar plot for rain first (in the background)
        ax.bar(month_data['Date'], month_data['rain_mm'], 
               width=1.0, label='Rain (mm)', color='skyblue', alpha=0.5)

        # Line for temperature (in the foreground)
        ax.plot(month_data['Date'], month_data['temperature_2m'], 
                label='Avg Temp (°C)', color='orange', linewidth=2)

        # Optional: uncomment to include wind speed too
        ax.plot(month_data['Date'], month_data['wind_speed_10m'], 
                label='Wind Speed (m/s)', color='green', linestyle='--')

        ax.set_title(f'Month: {month_names[month]}')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))
        ax.tick_params(axis='x', rotation=45)
        ax.set_ylabel('Values')
        ax.legend()

    plt.tight_layout()
    plt.show()

# Call the function to plot the weather data
plot_weather_data(weather)


ValueError: cannot assemble the datetimes: day is out of range for month, at position 59. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [49]:
print(weather.iloc[59][['day', 'month', 'Year']])
print(weather.iloc[59])

day         3.0
month       1.0
Year     2026.0
Name: 59, dtype: float64
temperature_2m       9.6520
rain                 0.0000
wind_speed_10m      23.0653
month                1.0000
day                  3.0000
hour                11.0000
Year              2026.0000
Name: 59, dtype: float64


In [12]:
# build a randomised customer order dataframe
orders = pd.DataFrame(columns=['ID', 'city', 'month', 'day', 'hour', 'minute', 'prep_time', 'customer', 'status'])

order_id = 0
day_of_year = 0

def generate_order_time(time_block=None):
    if time_block == 'morning':
        hour = np.random.randint(9, 12)
    elif time_block == 'lunch':
        hour = np.random.randint(12, 15)
    elif time_block == 'afternoon':
        hour = np.random.randint(15, 18)
    elif time_block == 'evening':
        hour = np.random.randint(18, 21)
    else:
        hour = np.random.randint(9, 21)
    minute = (np.random.randint(0, 12) * 5)  # gives 5 minute intervals
    return hour, minute

# AND NOW FOR THE FUN PART - GENERATING ORDERS :-)
for m in range(1,13):

    # use weather data to determine the day of the week number of days in the month
    weather_month = weather[weather['Month'] == m]
    days_in_month = weather_month['Day'].max()

    for d in range(1, days_in_month + 1):

        # what is the day of the week - assuming Jan 1st is a Monday - determine the base order probability
        day_of_week = day_of_year % 7
        if day_of_week < 5:     # weekday
            order_prob = 0.15
        elif day_of_week == 5:  # saturday
            order_prob = 0.15 * 1.2
        elif day_of_week == 6:  # sunday
            order_prob = 0.15 * 0.8

        # what is the average temperature for the day
        weather_day = weather_month[weather_month['Day'] == d]
        avg_temp = weather_day['DryBulbTemperature_C'].mean()
        avg_rain = weather_day['synthetic_rain_mm'].mean()

        if avg_temp < 10:
            # cold day
            if avg_rain > 0:
                # rainy day
                order_prob *= 1.2
            else:
                # dry day - but still reluctant to go out
                order_prob *= 1.1
            
        elif avg_temp < 20:
            # mild day
            if avg_rain > 0:
                # rainy day - mild and wet is unpleasant
                order_prob *= 1.1
            else:
                # dry day
                order_prob *= 1.0
            
        elif avg_temp < 30:
            # warm day
            if avg_rain > 0:
                # rainy day
                order_prob *= 1.0
            else:
                # dry day - might actually enjoy going out
                order_prob *= 0.95

        else:
            # hot day
            if avg_rain > 0:
                # rainy day
                order_prob *= 1.0
            else:
                # dry day - brutal heat
                order_prob *= 1.3


        # look to each location
        for i in range(1, 101):

            # weekday orders tend to follow a distribution with peaks at lunch and dinner times
            if day_of_week < 5:
                    
                # with probability 0.15, create a random order - (in a real-world scenario, this would be replaced with actual order data) - from a database or API
                if np.random.rand() < order_prob:
                
                    order_seed = np.random.rand() * 5.5
                    if order_seed < 1:
                        # morning orders are placed at random times between 8am and 10am
                        time_block = 'morning'
                    elif order_seed < 2.5:
                        # lunch orders are placed at random times between 12pm and 2pm
                        time_block = 'lunch'
                    elif order_seed < 3.5:
                        # afternoon orders are placed at random times between 3pm and 5pm
                        time_block = 'afternoon'
                    else:
                        # evening orders are placed at random times between 6pm and 8pm
                        time_block = 'evening'
            
                    order_hour, order_minute = generate_order_time(time_block)
                    order = pd.DataFrame({
                        'ID': order_id,
                        'city' : city,
                        'month' : m,
                        'day': d,
                        'hour': order_hour,
                        'minute': order_minute,
                        'prep_time': np.random.choice([1,5,10,15]),  # random prep time between 1 and 15 minutes
                        'customer': i,
                        'status': 'open'
                    }, index=[0])
                    order_id += 1
                    orders = pd.concat([orders, order], ignore_index=True)

            elif day_of_week == 5:
                # saturdays are busier
                if np.random.rand() < order_prob:
                    
                    order_hour, order_minute = generate_order_time()
                    order = pd.DataFrame({
                        'ID': order_id,
                        'city' : city,
                        'month' : m,
                        'day': d,
                        'hour': order_hour,
                        'minute': order_minute,
                        'prep_time': np.random.choice([1,5,10,15]),  # random prep time between 1 and 15 minutes
                        'customer': i,
                        'status': 'open'
                    }, index=[0])
                    order_id += 1
                    orders = pd.concat([orders, order], ignore_index=True)
                
            elif day_of_week == 6:
                # sundays are less busy
                if np.random.rand() < order_prob:
                    order_hour, order_minute = generate_order_time()
                    order = pd.DataFrame({
                        'ID': order_id,
                        'city' : city,
                        'month' : m,
                        'day': d,
                        'hour': order_hour,
                        'minute': order_minute,
                        'prep_time': np.random.choice([1,5,10,15]),  # random prep time between 1 and 15 minutes
                        'customer': i,
                        'status': 'open'
                    }, index=[0])
                    order_id += 1
                    orders = pd.concat([orders, order], ignore_index=True)
                
        day_of_year += 1


orders

,ID,city,month,day,hour,minute,prep_time,customer,status
0,0,Cambridge,1,1,13,30,1,2,open
1,1,Cambridge,1,1,16,55,10,18,open
2,2,Cambridge,1,1,20,40,15,23,open
3,3,Cambridge,1,1,19,25,10,25,open
4,4,Cambridge,1,1,17,25,15,27,open
...,...,...,...,...,...,...,...,...,...
6358,6358,Cambridge,12,31,12,15,10,82,open
6359,6359,Cambridge,12,31,13,15,10,84,open
6360,6360,Cambridge,12,31,11,25,10,86,open
6361,6361,Cambridge,12,31,16,15,5,89,open


In [13]:



# create a pack of robots
pack = []
pack.append(DeliveryRobot(name="Balto"))
pack.append(DeliveryRobot(name="Togo"))
pack.append(DeliveryRobot(name="Blackie"))
pack.append(DeliveryRobot(name="Fox"))

# display the pack
for robot in pack:
    print(robot)


Balto - SoC: 1.0, Status: IDLE
Togo - SoC: 1.0, Status: IDLE
Blackie - SoC: 1.0, Status: IDLE
Fox - SoC: 1.0, Status: IDLE


In [17]:
def calculate_distance(customer_id):
    """
    Calculate the Euclidean distance between two locations.
    """
    x1, y1 = locations.iloc[0]['x'], locations.iloc[0]['y']
    x2, y2 = locations.iloc[customer_id]['x'], locations.iloc[customer_id]['y']

    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calculate_slopes(customer_id, distance):
    """
    Calculate the slope between store and order location.
    """

    if distance == 0:
        return 0.0, 0.0

    # Calculate the slope percentages for both directions
    e0 = locations.iloc[0]['elevation']
    e1 = locations.iloc[customer_id]['elevation']
    print(f'store elevation {e0}, customer elevation {e1}')
    return ((e1 - e0) / distance) * 100.0, ((e0 - e1) / distance) * 100.0,

def terrain_energy_factors(slope_out, slope_back):
    """
    Calculate the energy factor based on slope percentage.
    """
    if slope_out > 0:
        terrain_factor_out = 1.0 + 0.03 * slope_out  # increases with uphill
    else:
        terrain_factor_out = max(0.85, 1.0 + 0.02 * slope_out)  # energy saving, capped

    if slope_back > 0:
        terrain_factor_back = 1.0 + 0.03 * slope_back  # increases with uphill
    else:
        terrain_factor_back = max(0.85, 1.0 + 0.02 * slope_back)  # energy saving, capped

    return terrain_factor_out, terrain_factor_back

def calculate_weather_factor(weather_info):
    """
    Guesstimate the weather based impact on the trip - input is a list of weather dataframes
    """
    avg_temp = np.mean(weather_info['DryBulbTemperature_C'])
    avg_rain = np.mean(weather_info['synthetic_rain_mm'])

    # Base weather factor
    weather_factor = 1.0

    # Adjust for temperature - cold weather slows us down
    if avg_temp < 0:
        weather_factor *= 0.95
    elif avg_temp < 10:
        weather_factor *= 0.99

    # Rain effects — ignore anything under 2.5mm
    if avg_rain > 7.5:
        weather_factor *= 0.9
    elif avg_rain > 5.0:
        weather_factor *= 0.95
    elif avg_rain > 2.5:
        weather_factor *= 0.98
    # <2.5 → no adjustment
    
    return weather_factor

def speed_adjustment(slope_percent, base_speed_mps=1.2, weather_factor=1.0):
    """
    Adjusts speed based on slope and weather factor.
    Uphill slows you down, downhill helps a little.  Rain slows you down.
    """
    
    # Convert slope percentage to degrees
    slope_degrees = math.atan(slope_percent / 100) * (180 / math.pi)
    adjusted_speed = base_speed_mps

    # Adjust speed based on slope
    if slope_degrees > 0:
        adjusted_speed = base_speed_mps * (1 - slope_degrees / 45)  # Slower uphill
    else:
        adjusted_speed = base_speed_mps #* (1 + abs(slope_degrees) / 90)  # Faster downhill
    adjusted_speed = max(0.1, adjusted_speed * weather_factor)  # Adjust for weather

    return adjusted_speed

def speed_adjustments(slope_out_percent, slope_back_percent, base_speed_mps=1.2, weather_factor=1.0):
    """
    Adjusts speed based on slope and weather factor.
    Uphill slows you down, downhill helps a little.  Rain slows you down.
    """
    
    # Convert slope percentage to degrees
    slope_out_degrees = math.atan(slope_out_percent / 100) * (180 / math.pi)
    slope_back_degrees = math.atan(slope_back_percent / 100) * (180 / math.pi)
    adjusted_speed_out = base_speed_mps
    adjusted_speed_back = base_speed_mps
    # Adjust speed based on slope
    if slope_out_degrees > 0:
        adjusted_speed_out = base_speed_mps * (1 - slope_out_degrees / 45)  # Slower uphill
    else:
        adjusted_speed_out = base_speed_mps #* (1 + abs(slope_out_degrees) / 90)  # Faster downhill
    adjusted_speed_out = max(0.1, adjusted_speed_out * weather_factor)  # Adjust for weather

    if slope_back_degrees > 0:
        adjusted_speed_back = base_speed_mps * (1 - slope_back_degrees / 45)
    else:
        adjusted_speed_back = base_speed_mps #* (1 + abs(slope_back_degrees) / 90)
    adjusted_speed_back = max(0.1, adjusted_speed_back * weather_factor)

    return adjusted_speed_out, adjusted_speed_back

# Example usage
weather_info = weather[(weather['Month'] == 1) & (weather['Day'] == 1) & (weather['Hour'] == 12)]
weather_info = pd.concat([weather_info, weather[(weather['Month'] == 1) & (weather['Day'] == 1) & (weather['Hour'] == 13)]])
w_factor = calculate_weather_factor(weather_info)
print(f"Weather factor: {w_factor:.2f}")

# Example usage
customer = 38
dist = calculate_distance(customer)
slope_out, slope_back = calculate_slopes(customer, dist)
energy_factor_out, energy_factor_back = terrain_energy_factors(slope_out, slope_back)
speed_out, speed_back = speed_adjustments(slope_out, slope_back, w_factor)
print(f"Distance to customer {customer}: {dist:.2f} m over a slope of {slope_out:.2f} %, resulting in a slope energy factor of {energy_factor_out:.2f} out and {energy_factor_back:.2f} back")
print(f"Speed adjustment for slopes: {speed_out:.2f} m/s out and {speed_back:.2f} m/s back")


def estimated_time_to_customer(customer_id, weather_info):
    """
    Estimate time to customer based on distance, slope, and weather.
    """
    # Calculate distance and slopes
    distance = calculate_distance(customer_id)
    slope_out, _ = calculate_slopes(customer_id, distance)

    # Calculate weather factor
    weather_factor = calculate_weather_factor(weather_info)

    # Adjust speed
    speed_out, _ = speed_adjustments(slope_out, slope_back, weather_factor=weather_factor)

    # Calculate time to customer
    time_to_customer = (distance / speed_out) / 60

    return time_to_customer

print(f"Estimated time to customer {customer}: {estimated_time_to_customer(customer, weather_info):.2f} minutes")


def estimated_energy_consumption(customer_id, weather_info):
    """
    Estimate energy consumption for a trip to a customer.
    """
    # Calculate distance and slopes
    distance = calculate_distance(customer_id)
    distance_km = distance / 1000
    slope_out, slope_back = calculate_slopes(customer_id, distance)

    # Calculate weather factor
    weather_factor = calculate_weather_factor(weather_info)

    energy_factor_out, energy_factor_back = terrain_energy_factors(slope_out, slope_back)

    # Calculate energy consumption
    estimated_energy_required = (distance_km * weather_factor * energy_factor_out) + (distance_km * weather_factor * energy_factor_back)  

    return estimated_energy_required

print(f"Estimated energy consumption for customer {customer}: {estimated_energy_consumption(customer, weather_info):.2f} Wh")

def calculate_energy_consumption(distance, slope, weather_info):
    """
    Calculate energy consumption for a trip to a customer.
    """
    # Calculate distance and slopes
    distance_km = distance / 1000

    # Calculate weather factor
    weather_factor = calculate_weather_factor(weather_info)

    energy_factor, _ = terrain_energy_factors(slope, slope)
    # Calculate energy consumption
    estimated_energy_required = (distance_km * weather_factor * energy_factor)  

    return estimated_energy_required

print(calculate_energy_consumption(900, 3.4, weather))

def calculate_time_offset(h, m, offset):
    """
    given a time in hours and minutes, and an offset in minutes, what is the resulting time?
    """
    if offset > 0:
        # greater than an hour
        if m+offset >= 60:
            temp = h*60 + m + offset
            new_h = (temp // 60)
            new_m = (temp % 60)
        else:
            new_h = h
            new_m = m + offset
    else:
        print("Negative offset not supported")
        return h, m
    
    return new_h, new_m


print(calculate_time_offset(9, 13, 134))

Weather factor: 0.99
store elevation 9.77, customer elevation 19.98
Distance to customer 38: 1060.66 m over a slope of 0.96 %, resulting in a slope energy factor of 1.03 out and 0.98 back
Speed adjustment for slopes: 0.98 m/s out and 0.99 m/s back
store elevation 9.77, customer elevation 19.98
Estimated time to customer 38: 15.06 minutes
store elevation 9.77, customer elevation 19.98
Estimated energy consumption for customer 38: 2.11 Wh
0.9818820000000001
(11, 27)


In [14]:
# day loop
global current_hour, current_minute, month, day

# set the initial time
month = 1
day = 1

current_hour = 0
current_minute = 0

# these are the accepted taskings for the robots
taskings = []

# is called in a loop - for m in range(0, 1440):
def dispatch_logic(month, day, current_hour, current_minute):

    print(f"Dispatch logic at {current_hour}:{current_minute:02d} on {day}/{month}")

    # check the weather data for the last hour, the current hour and the next hour
    weather_info = weather[(weather['Month'] == month) & (weather['Day'] == day) & (weather['Hour'] == current_hour-1)]
    weather_info = pd.concat([weather_info, weather[(weather['Month'] == month) & (weather['Day'] == day) & (weather['Hour'] == current_hour)]])
    weather_info = pd.concat([weather_info, weather[(weather['Month'] == month) & (weather['Day'] == day) & (weather['Hour'] == current_hour+1)]])
    weather_factor = calculate_weather_factor(weather_info)

    # check for new orders
    new_orders = orders[(orders['month'] == month) & (orders['day'] == day) & (orders['hour'] == current_hour) & (orders['minute'] == current_minute)]

    if len(new_orders) > 0:
        
        print(f"New orders at {current_hour}:{current_minute:02d} - {len(new_orders)} new orders")
        for i, order in new_orders.iterrows():

            customer_id = order['customer']
            prep_time = order['prep_time']
            # distance
            dist_to_customer = calculate_distance(customer_id)
            slope_out, slope_back = calculate_slopes(customer_id, dist_to_customer)
            speed_out, speed_back = speed_adjustments(slope_out, slope_back, weather_factor=weather_factor)

            # time to customer - can we get there in time?
            expected_journey_time_to_customer = (dist_to_customer / speed_out) / 60  # in minutes
            if expected_journey_time_to_customer > 60:
                print(f"Order {order['ID']} for customer {customer_id} at location ({locations.loc[customer_id, 'x']}, {locations.loc[customer_id, 'y']}) - expected journey time: {expected_journey_time_to_customer:.2f} minutes - cannot accept order")
                order['status'] = 'rejected - cannot meet time'
            else:

                # expected arrival time at customer
                expected_arrival_time_at_customer = current_hour * 60 + current_minute + expected_journey_time_to_customer

                # energy expenditure for there and back
                expected_energy_for_order = estimated_energy_consumption(customer_id, weather_info)

                # are there untasked robot(s) with sufficient SoC?
                available_robots = [robot for robot in pack if robot.status == 'IDLE' and robot.battery.get_current_kwh() > expected_energy_for_order]
        
                # is a robot returning in time with sufficient time and expected battery charge to meet order?
                for robot in pack:
                    if robot.status == 'ON_TASK_RETURNING':
                        # check if the robot can return to base and then go to the customer
                        returning_task = robot.get_task_history()[-1]
                        

                        dist_to_base = calculate_distance(locations.loc[0, ['x', 'y']], robot.get_current_location())
                        _, slope_back = calculate_slopes(returning_task.get_order_id(), dist_to_base)
                        # energy expenditure for home and then there and back
                        expected_energy_for_return = calculate_energy_consumption(dist_to_base, slope_back, weather_factor)
                        expected_energy_for_total = expected_energy_for_order + expected_energy_for_return
                        
                        # time required for home and immediate load (1 min) and then there 
                        expected_time_for_return = (dist_to_base / speed_back) / 60
                        expected_extended_task_time = expected_time_for_return + 1 + expected_journey_time_to_customer
                        # this expected time is WRONG
                        expected_arrival_time_at_customer = current_hour * 60 + current_minute + expected_extended_task_time

                        if expected_extended_task_time < 60 and robot.battery.get_current_charge() > expected_energy_for_total:
                            available_robots.append(robot)

                # is a robot being charged such that it could reach sufficient SoC and meet window
                # for robot in pack:
                #     if robot.status == 'CHARGING':
                # TO DO        
                    

                if len(available_robots) == 0:
                    print(f"Order {order['ID']} for customer {customer_id} at location ({locations.loc[customer_id, 'x']}, {locations.loc[customer_id, 'y']}) - expected journey time: {expected_journey_time_to_customer:.2f} minutes - no robots available")
                    order['status'] = 'rejected - no robots available'
                else:
                    print(f"Order {order['ID']} for customer {customer_id} at location ({locations.loc[customer_id, 'x']}, {locations.loc[customer_id, 'y']}) - expected journey time: {expected_journey_time_to_customer:.2f} minutes - {len(available_robots)} robots available")
                    order['status'] = 'accepted - robot assigned'

                    # randomly select a robot from the available robots
                    robot = np.random.choice(available_robots)
                    task_start_hour, task_start_minute = calculate_time_offset(current_hour, current_minute, prep_time)
                    task = Task(order['ID'], robot.name, task_start_hour, task_start_minute)
                    taskings.append(task)                 
                                
                    print(f"Order {order['ID']} for customer {order['customer']} at location ({locations.loc[order['customer'], 'x']}, {locations.loc[order['customer'], 'y']}) - distance: {dist_to_customer:.2f} - tasked to {robot.name} expected arrival time: {expected_arrival_time_at_customer:.2f} minutes")

# main loop
# for m in range(1, 13):
#     for d in range(1, 29):

for m in range(1,2):
    for d in range(1,2):

        # check if we have reached the end of the month
        if d == 28 and m == 2:
            break     # check if we have reached the end of the month
        if d == 28 and m == 2:
            break
        elif d == 30 and m in [4, 6, 9, 11]:
            break
        elif d == 31 and m in [1, 3, 5, 7, 8, 10, 12]:
            break

        # set the month and day
        month = m
        day = d

        print(f"Processing month: {month}, day: {day}")

        current_hour = 0
        current_minute = 0

        # loop through the hours and minutes of the day
        for min in range(0, 1440):
            print(f"Processing minute: {min}")
            
            if min % 60 == 0:
                current_hour += 1
                current_minute = 0
            else:
                current_minute += 1
            print(f"Current time: {current_hour}:{current_minute:02d} on {day}/{month}")

            # check weather data for the current hour
            weather_now = weather[(weather['Month'] == month) & (weather['Day'] == day) & (weather['Hour'] == current_hour)]
            if weather_now['synthetic_rain_mm'].values[0] > 10:
                print(f"Heavy rain at {current_hour}:{current_minute:02d} - too dangerous to operate")
            else: 
                # dispatch logic
                dispatch_logic(month, day, current_hour, current_minute)


            # # have we reached the end of the day?
            # if current_hour == 21 and current_minute == 0:
            #     print(f"End of day reached: {current_hour}:{current_minute:02d} on {day}/{month}")
            #     break


Processing month: 1, day: 1
Processing minute: 0
Current time: 1:00 on 1/1
Dispatch logic at 1:00 on 1/1
Processing minute: 1
Current time: 1:01 on 1/1
Dispatch logic at 1:01 on 1/1
Processing minute: 2
Current time: 1:02 on 1/1
Dispatch logic at 1:02 on 1/1
Processing minute: 3
Current time: 1:03 on 1/1
Dispatch logic at 1:03 on 1/1
Processing minute: 4
Current time: 1:04 on 1/1
Dispatch logic at 1:04 on 1/1
Processing minute: 5
Current time: 1:05 on 1/1
Dispatch logic at 1:05 on 1/1
Processing minute: 6
Current time: 1:06 on 1/1
Dispatch logic at 1:06 on 1/1
Processing minute: 7
Current time: 1:07 on 1/1
Dispatch logic at 1:07 on 1/1
Processing minute: 8
Current time: 1:08 on 1/1
Dispatch logic at 1:08 on 1/1
Processing minute: 9
Current time: 1:09 on 1/1
Dispatch logic at 1:09 on 1/1
Processing minute: 10
Current time: 1:10 on 1/1
Dispatch logic at 1:10 on 1/1
Processing minute: 11
Current time: 1:11 on 1/1
Dispatch logic at 1:11 on 1/1
Processing minute: 12
Current time: 1:12 on 1/

In [16]:
taskings

[Task(order_id=12, robot_id=Togo, start_time=9:45),
 Task(order_id=16, robot_id=Blackie, start_time=11:16),
 Task(order_id=10, robot_id=Togo, start_time=11:50),
 Task(order_id=15, robot_id=Fox, start_time=13:46),
 Task(order_id=2, robot_id=Balto, start_time=17:50),
 Task(order_id=14, robot_id=Blackie, start_time=19:0),
 Task(order_id=3, robot_id=Fox, start_time=19:10),
 Task(order_id=13, robot_id=Blackie, start_time=20:25),
 Task(order_id=6, robot_id=Blackie, start_time=20:40)]